# Wichtige Funktionen zur Auswertung von Versuchen. 
Diese Vorlage kann kopiert werden und dann für den jeweiligen Versuch genutzt werden. 

Zunächst soll die Versuchsummer des jeweiligen Versuches angegeben werden, damit das automatische Speichern der Grafiken in das richtige Verzeichnis funktioniert.

Dieses Dokument ist an vielen Stellen (stark) an *Luca Hafner* orientiert.

## Versuchsnummer:

In [1]:
versuchsnummer = "000"

## Import aller wichtigen Libaries

In [38]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.transforms as transforms
%matplotlib inline

import numpy as np
from numpy import exp, sqrt, log, pi

from scipy import odr
import scipy.optimize
from scipy.optimize import curve_fit
from scipy.stats import chi2
from scipy.stats import poisson
from scipy.integrate import quad
from scipy.signal import find_peaks
from scipy.signal import argrelextrema, argrelmin, argrelmax
from scipy.special import factorial

import os
import os.path

import pandas as pd
import csv
import re

from typing import List

import math
from decimal import Decimal, ROUND_HALF_UP, getcontext

import pylab as py
from IPython.display import display, Math, Latex, HTML

import sympy as sp
from sympy import separatevars

### Erstellen der dedizierten Dartei zum Speichern der Werte für **LATEXT**

Folgender Code sollte einmal ausgeführt werden, damit direkt die Datei zum Speichern erstellt werden kann. Unbedingt daran denken, die Versuchsnummer anzupassen, die Werte anderer Versuche könnten ansonsten verloren gehen. Es gibt jedoch eine sicherheits Kopie. 

Diese beinhaltet:
* Formel, sowie Formel nach gff
* Berechnete Werte und deren Fehler
    * Wert + Fehler
    * Tabellen Export

In [3]:
def create_tex_result_values(fileName = "python-results.sty"):
    """
    Diese Method erstellt automatisch eine tex-Datei, in dem Messwerte bzw. deren Ergebnisse, Tabellen weiteres als variable gespeichert werden, die hier in diesem Python-code bestimmt werden.
    Die File wird unter *Versuche/${versuchsnummer}$/Auswertung/python-results.tex* zufinden sein. Am einfachsten ist es jedoch die Parameter frei zuhalten, da ansonsten auch das Verzeichnis in der *main.tex* 
    angepasst werden muss. 

    Die Python-File muss im Python-Ordner des jeweiligen Versuches liegen!

    Parameter
    ----------
    **fileName**: 
        neuer Name, falls die Datei besonders heißen soll.
    """
    # Erstellung der file

    path = "../Auswertung/" + fileName

    if os.path.isfile(path):
        pass
    else:
        with open(path, 'w') as file:
            file.write("% Dies ist eine automatisch generierte Datei. Hier werden automatisiert Variablen fuer Formeln, Ergebnisse und Tabellen erstellt. \n% Bitte nicht in diese Datei schreiben. Informationen koennten geloescht oder nicht richtig verarbeitet werden. \n\n%  _          _   _       _______      \n% | |        | | ( )     |__   __|\n% | |     ___| |_|/ ___     | | _____  __\n% | |    / _ \ __| / __|    | |/ _ \ \/ /\n% | |___|  __/ |_  \__ \    | |  __/>  < \n% |______\___|\__| |___/    |_|\___/_/\_\ \n\n\n") 

        with open(path + "-BackUp.sty", 'w') as bFile:
            bFile.write("% Dies ist eine automatisch generierte Datei. Hier wird dediziert ein Back-Up erstellt, damit Werte nicht verloren gehen. \n\n%  _          _   _       _______      \n% | |        | | ( )     |__   __|\n% | |     ___| |_|/ ___     | | _____  __\n% | |    / _ \ __| / __|    | |/ _ \ \/ /\n% | |___|  __/ |_  \__ \    | |  __/>  < \n% |______\___|\__| |___/    |_|\___/_/\_\ \n\n\n") 

    # print(filePath + "/" + fileName)
    # print(versuchsnummer)

    return path

create_tex_result_values()
pyPath = create_tex_result_values()



<>:22: SyntaxWarning: invalid escape sequence '\ '
<>:25: SyntaxWarning: invalid escape sequence '\ '
<>:22: SyntaxWarning: invalid escape sequence '\ '
<>:25: SyntaxWarning: invalid escape sequence '\ '
C:\Users\finnz\AppData\Local\Temp\ipykernel_3520\4151508109.py:22: SyntaxWarning: invalid escape sequence '\ '
  file.write("% Dies ist eine automatisch generierte Datei. Hier werden automatisiert Variablen fuer Formeln, Ergebnisse und Tabellen erstellt. \n% Bitte nicht in diese Datei schreiben. Informationen koennten geloescht oder nicht richtig verarbeitet werden. \n\n%  _          _   _       _______      \n% | |        | | ( )     |__   __|\n% | |     ___| |_|/ ___     | | _____  __\n% | |    / _ \ __| / __|    | |/ _ \ \/ /\n% | |___|  __/ |_  \__ \    | |  __/>  < \n% |______\___|\__| |___/    |_|\___/_/\_\ \n\n\n")
C:\Users\finnz\AppData\Local\Temp\ipykernel_3520\4151508109.py:25: SyntaxWarning: invalid escape sequence '\ '
  bFile.write("% Dies ist eine automatisch generierte D

## Definition der Funktionen

---
## Table of Contents:
* [Fehlerrechnung einer gegebenen Formel](#Fehlerrechnun-einer-gegebenen-Formel)
* [Automatisierter Latex-Export](#Automatisierter-Latex-Export)
    * [Export der Latex Formel](#Export-der-Latex-Formel)
    * [Ergebnisse plus Fehler](#Ergebnisse-plus-Fehler)
    * [Export als Tabelle](#Export-als-Tabelle)
---

## Fehlerrechnung einer gegebenen Formel

**Gauss Fehler Fortpflanzung (GFF)**:

Die *Methode gff* nimmt zwei Argumente, zum einen eine *sympy (sp)* Funktion, zum anderen einen Array aller fehlerbehafteter Größen.

In [4]:
def gff(func, errPronePar):
    """
    Führt standardgemaess die Gaussian Error Propergation aus.
    """
    # Variablen definieren
    error = 0
    errProneParamters = []
    for errPar in errPronePar:
        d = sp.symbols('d' + errPar.name)
        partial = sp.diff(func, errPar) * d  # Die Funktion wird nach der fehlerbehafteten Variable abgeleitet
        error = error + partial**2 # Fehler werden quadratisch aufsummiert
        errProneParamters.append((errPar,d))
    absolut_err=sp.simplify(sp.sqrt(error),rational = True)             
    relativ_err=sp.simplify(sp.sqrt(error/func**2),rational = True)
    return absolut_err, relativ_err, errProneParamters

Beispiel gff:

In [5]:
# Drei Messgrößen x, y und z werden definiert.
x = sp.Symbol('x')
y = sp.Symbol('y')
z = sp.Symbol('z')

params = [x, y, z]

# Darstellen einer Funktion
f = x**x

# Unsicherheiten (bspw. Ablesefehler)
dx_val = 0.05  
dy_val = 0.02 
dz_val = 0.10  


# Ausführen von gff
abs_err, rel_err, param_symbols = gff(f, params)

print("Absoluter Fehler (delta f):")
sp.pprint(abs_err)

print("\nRelativer Fehler (delta f / f):")
sp.pprint(rel_err)

Absoluter Fehler (delta f):
   ________________________
  ╱   2  2⋅x             2 
╲╱  dx ⋅x   ⋅(log(x) + 1)  

Relativer Fehler (delta f / f):
   ___________________
  ╱   2             2 
╲╱  dx ⋅(log(x) + 1)  


### Export einer Funktion in Latex

Die folgende Funktion transformiert eine Formel in Latex. Diese Formel kann entweder hier im Notebook einfach kopiert werden oder als Variable in der *pyton-results.tex*.

In [6]:
def function_to_latex(f, texVarName, texCom):
    """
    Zeigt die Formel als gerenderte Math-Darstellung und darunter
    den Latex-Quelltext, der per Button kopiert werden kann. (Für leichtere Benutzung als HTML).
    
    Zudem wird die Latexformel als Variable in Latex gespeichert.

    Parameters
    ----------
    **f** : sympy function

    **texVarName** : Einzigartiger Name der Variablen. Dieser wird zum überschreiben alter Formel gebraucht.

    **texCom** : setzt den newcommand-Kürzel für latex fest. Setzte kein Backslash! Auch texCom muss einzigartig gesetzt werden.
    """
    # print("Die gegebene Funktion lautet: \n")
    display(Math(sp.latex(f, long_frac_ratio=2).replace('d__', r'\Delta ')))

    # print("Hier ist der dazugehörige Latex code: \n")
    latex_str = sp.latex(f, long_frac_ratio=2).replace('d__', r'\Delta ')
    html = f"""
    <div style="margin-top:0.5em;">
        <code id="latex-code-{id(f)}">
            {latex_str}
        </code>
        <button onclick="
            const tex_as_txt = document.getElementById('latex-code-{id(f)}').innerText;
            navigator.clipboard.writeText(tex_as_txt)
        " style="
            margin-left:8px;
            padding:2px 6px;
            cursor:pointer;
        ">
            Kopieren
        </button>
    </div>
    """
    display(HTML(html))


    # Hinzufuegen bzw. Ueberschreieben der Formel in die Sammlung
    with open(pyPath, 'r') as file:
        lines = file.readlines()
    found = False

    with open(pyPath, 'r') as file:
        for lineNum,  line in enumerate(lines, 1):
            if texVarName in line: # Checkt, ob der Variablen Name bereits vergeben ist.
                print(f'{texVarName} is at line {lineNum}') 
                lines[lineNum] = "\\newcommand{\\" + texCom + "}{" + latex_str + "}\n"
                found = True

    if not found:
        print("Die neue Funktion wurde hinzugefügt")
        with open(pyPath, 'w') as file:
            file.writelines(lines) # Schreibt den alten Stand
            file.write("\n%" + texVarName + "\n")
            file.write("\\newcommand{\\" + texCom + "}{" + latex_str + "}\n\n")
    else:
        print("Die alte Funktion wurde erfolgreich überschrieben.")
        with open(pyPath, 'w') as file:
            file.writelines(lines)

Beispiel function_to_latex:

In [7]:
f = y**2

line = function_to_latex(f, "Parabel", "par")

<IPython.core.display.Math object>

Die neue Funktion wurde hinzugefügt


### Ergebnisse plus Fehler

In [467]:
def round_sig_digs(errVal: float, val: float=None, round_to_dig: int=None):
    """
    Function that rounds error value accoring the the standardised format:
    the function will search for the first digit that is greater than 2. If it found such a dig it will round this dig. It must not be after the decimal point!
    It will than present the value according to the errVale, thus they have the same amount of digs.

    Parameter
    ---------
    **errValue** : float to be rounded to significant digit.

    **value** : float that will be rounded accoring to the errValue.

    **round_to_dig** : int to wich will be rounded

    Returnes
    --------
    rounded : float
    """

    errVal = abs(errVal)

    # Convert to string with 20 digs. Delets 0 at the end
    errVal_str = f'{errVal}'.rstrip('0')
    if '.' not in errVal_str:
        errVal_str += '.0'

    val_str = f'{val}'.rstrip('0')
    if '.' not in val_str:
        val_str += '.0'

    rounded_err_int= ""
    rounded_err_frac = ""

    num_of_digs_int = None
    err_int, err_frac = errVal_str.split('.')

    val_int, val_frac = errVal_str.split('.')


    rounded_int= ""
    rounded_frac = ""

    for pos, dig in enumerate(err_int, 1):
        if int(dig) < 3:
            # print(f"{dig} is no sig dig")
            pass
        else:
            num_of_digs_int = pos
            # print(f"{dig} is sig dig at position {num_of_digs_int}!")
            round_to_dig = len(err_int) - pos
            rounded_err_int = np.round(int(err_int), -round_to_dig)
            rounded_int = int(np.round(val, - round_to_dig))
            print(f"Value: {rounded_int}")
            print(F"Err: {rounded_err_int}")
            break

    if num_of_digs_int == None:
        # print("Es wurde keine sig dig im int gefunden!")
        for pos_frac, dig_frac in enumerate(err_frac, 1):
            if int(dig_frac) < 3:
                # print(f"{dig_frac} is no sig dig")
                pass
            else:
                # print(f"{dig_frac} is sig dig at position {num_of_digs_frac} in fractorial part!")
                round_to_dig = len(err_frac) - pos_frac
                rounded_err_frac = np.round(int(err_frac), - round_to_dig)
                rounded_int = np.round(val, pos_frac)
                print(f"Value: {rounded_int}")
                print(F"Err: {err_int}.{rounded_err_frac}")
                break
            if (('1' or '2' in list(err_frac)) and (len(set(list(err_frac))) == 1)) or ((('1' and '2') in list(err_frac)) and (len(set(list(err_frac))) == 2)):
                print("Es existiert keine sig dig")
                rounded_int = str(val) + (len(err_frac)-len(val_frac)) * "0"
                print(f"Value: {rounded_int}")
                print(f"Err: {errVal}")
                break

            



    # # Check if integer_part contains sig dig
    # for pos, dig in enumerate(int_part):
    #     # print(pos,dig)
    #     if dig.isdigit and int(dig) > 2:
    #         print("Signifikante Stelle erreicht: " + dig)
    #         num_of_digs = pos + 1 # Position der signifikanten Stelle
    #         # Checken, ob die folge Stelle kleiner oder groesser gleich 5 ist

    #         # Checken, ob durch dezimalstellen gerunded werden muss
    #         if len(int_part) == 1 and int(frac_part[0]) >= 5:
    #             print("Aufrunden durch erste dezimal Stelle")
    #             rounded_dig = int(int_part[0]) + 1 # signifikante Stelle wird aufgerundend
    #             rounded_err_int_str = str(rounded_dig) + ".0" 
    #             # return rounded_int_part_str
    #             break

    #         elif len(int_part) == 1 and int(frac_part[0]) < 5:
    #             print("Abrunden durch erste dezimal Stelle")
    #             rounded_dig = int(int_part[0])
    #             rounded_err_int_str = str(rounded_dig) + ".0"
    #             # return rounded_int_part_str
    #             break

    #         elif int(int_part[num_of_digs]) >= 5:
    #             print("Aufrunden")
    #             digs_before_sig= int_part[0:num_of_digs-1] # Stellen, die nicht signifikant sind, sollen bleiben
    #             rounded_dig = int(int_part[num_of_digs-1]) + 1 # signifikante Stelle wird aufgerundend
    #             rounded_err_int_str = digs_before_sig + str(rounded_dig) + "0" * (len(int_part) - len(digs_before_sig) - 1)
    #             # return rounded_int_part_str
    #             break

    #         else:
    #             print("Abrunden")
    #             digs_before_sig= int_part[0:num_of_digs-1] # Stellen, die nicht signifikant sind, sollen bleiben
    #             rounded_dig = int_part[num_of_digs-1] # signifikante Stelle wird gleich gelassen
    #             rounded_err_int_str = digs_before_sig + str(rounded_dig) + "0" * (len(int_part) - len(digs_before_sig) - 1)
    #             # return rounded_int_part_str
    #             break
        
        

    #     elif dig.isdigit and int(dig) == (1 or 2):
    #         # print(f"{dig} ist nicht signifikant")
    #         pass

    #     # Signifikante Stelle liegt im dezimal bereich.
    #     else:
    #         rounded_err_int_str = int_part + '.'
    #         frac_part = frac_part + "0"
    #         print("Nur dezimal teil!")
    #         for pos, dig in enumerate(frac_part):
    #             if dig.isdigit and int(dig) > 2:
    #                 print("Signifikante Stelle erreicht: " + dig)
    #                 num_of_digs = pos + 1 # Position der signifikanten Stelle
    #                 # Checken, ob die folge Stelle kleiner oder groesser gleich 5 ist
    #                 if int(frac_part[num_of_digs]) >= 5:
    #                     print("Aufrunden")
    #                     digs_before_sig= frac_part[0:num_of_digs-1] # Stellen, die nicht signifikant sind, sollen bleiben
    #                     rounded_dig = int(frac_part[num_of_digs-1]) + 1 # signifikante Stelle wird aufgerundend
    #                     rounded_err_frac_str = digs_before_sig + str(rounded_dig)
    #                     # return rounded_frac_part_str
    #                     break
    #                 else:
    #                     print("Abrunden")
    #                     digs_before_sig= frac_part[0:num_of_digs-1] # Stellen, die nicht signifikant sind, sollen bleiben
    #                     rounded_dig = frac_part[num_of_digs-1] # signifikante Stelle wird gleich gelassen
    #                     rounded_err_frac_str = digs_before_sig + str(rounded_dig)
    #                     # return rounded_frac_part_str
    #                     break

    # print(f"Position der signifikanten stell: {num_of_digs}")
    # round_val = num_of_digs

    # rounded_err = rounded_err_int_str + rounded_err_frac_str
    # # round_val = None
    # print(rounded_err)
    # print(round(val, round_val))
    # print("We want: 1240 pm 30")

round_sig_digs(errVal=1.12, val=22438.4239)

Es existiert keine sig dig
Value: 22438.4239
Err: 1.12


In [9]:
def calc_with_err(func, errFunc, values):
    """
    Methode zum berechnen von Werten und deren Fehler.

    Parameter
    ----------
    **func** : sympy Funktion mit Parametern. 

    **errFunc** : die zu func gehöhrende sympy Fehler-Funktion nach gff

    **values** : 
        Werte, die in die Funktionen eingesetzt werden.
        Als array von Tupeln der Form [(a,da),(b,db),...] oder als array/liste [a,da,b,db,...] 
        (Reihenfolge muss die sein, in der die Argumente in der Funktion genommen werden)
    """

    #Falls der Input in mehrere Tupel aufgeteilt ist, werden diese zu einem Array zusammengefügt 
    if (np.ndim(values) != 1):                    
        values = np.concatenate(values)
    #print(values)
    result = func(*values[::2])
    uncertainty = errFunc(*values)
    return result, uncertainty

In [10]:
def pap(function, texVarName, texCom, params, data, params_without_error=[]):
    """
    Nimmt als imput eine Funktion und die Information welche Parameter fehlerbehaftet sind. Zudem 

    Parameter
    ----------
    **function** : 
        sympy Funktion mit Parametern. In diese werden die Messwerte eingesetzt.

    **texVarName** : 
        Einzigartiger Name der Variablen. Dieser wird zum überschreiben alter Formel gebraucht.

    **texCom** :     
        setzt den newcommand-Kürzel für latex fest. Setzte kein Backslash! Auch texCom muss einzigartig gesetzt werden.

    **params** : 
        Parameter der Funktion. Diese werden als Array von Sympy-Symbolen gebraucht. Bspw. [x, y, z]

    **data** :
        2D-Array mit den Messdaten, sodass die Zeilen die Form haben: [Parameter 1, Fehler Parameter 1, Parameter 2,...]
        Die Funktion wird zeilenweise angewandt. Wird kein Fehler für einen Parameter angenommen, kann diese Spalte entweder mit dem Wert 0 
        an die Funktion gegeben werden oder ganz weggelassen werden. Dann muss allerdings der betreffende Parameter bei params_without_error angegeben werden.

    **params_without_error** : 
        Alle Parameter zu denen kein Fehler explizit in den Daten angegeben ist. Dieser wird auf 0 gesetzt und kommt dann
        auch nicht in der Latex Form der Fehlerformel vor

    """
    # Expand data to include the uncertainty 0 for values with no assigned uncertainty
    # Hat zubeginn die Form: [[0. 0. 0. 0. 0. 0. ...]]. 
    # Bei jeder Iteration werden dann die Parameter und deren Fehler hinzugefügt: i=1 -> [[par1 errPar1 0. 0. 0. 0. ...]]
    exp_data = np.zeros((data.shape[0],data.shape[1]+len(params_without_error)))
    i = 0      # läuft durch die Parameter
    j = 0      # läuft durch die expanded data
    z = 0      # läuft durch die eingegebene data, also die Messwerte und deren Fehler
    # Läuft durch jeden Parameter und seinen Fehler
    while (i < len(params)):
        # Checkt, ob der Parameter Fehlerbehaftet ist. Wenn, dann wird an j-ter Stelle des exp_data der z-te Parameter aus data einegfügt.
        if (params[i] in params_without_error):
            exp_data[:,j] = data[:,z]
            i = i + 1
            j = j + 2
            z = z + 1
        else:
            exp_data[:,j] = data[:,z]
            exp_data[:,j+1] = data[:,z+1]
            i = i + 1
            j = j + 2
            z = z + 2
    # print(exp_data)

    # Create variable that stores parameters that have no assigned uncertainty    
    params_with_error = []
    j = 0
    for n in np.arange(0,len(params)):
        if not (params[n] in params_without_error):
            params_with_error.append(params[n])
            j = j + 1
    
    # Get the given function and error function as numpy functions
    f = sp.lambdify(params,function)
    absolut_err, relativ_err, parameters = gff(function,params) # Gauss Fehlerfortpflanzung
    err_abs = sp.lambdify(np.concatenate(parameters),absolut_err)

    # Calculate the results for each row of data
    results = np.zeros((data.shape[0],2))
    for n in np.arange(0,data.shape[0]):
        results[n,:] = calc_with_err(f, err_abs, exp_data[n,:])
        # print(calc_with_err(f, err_abs, exp_data[n]))
    
    if (len(results) < 10):
        print("Results:")
        print(results)

    # Substitutes 0 for the uncertainty of the parameters without error, so it doesnt show up in the Latex Code
    for p in params_without_error:
        absolut_err = absolut_err.subs('d'+p.name,0)
    for p in params_without_error:
        relativ_err = relativ_err.subs('d'+p.name,0)

    # Wiedergabe der Formeln in Latex
    function = sp.simplify(function,symbols = params, rational= True)
    function = sp.separatevars(function)
    
    print("gegebene Funktion:")
    function_to_latex(function, texVarName, texCom)

    print("Formel des absoluten Fehlers der gegebenen Funktion:")
    function_to_latex(absolut_err, "errAbs-" +  texVarName, "errAbs" +  texCom)

    print("Formel des relativen Fehlers der gegebenen Funktion:")
    function_to_latex(relativ_err, "errRel-" +  texVarName, "errRel" +  texCom)

    return(results)


Beispiel calc_with_err:

In [11]:
#Beispieleingabe für die Funktion "uncertainty" (Daten aus Jens' Fehlerrechner)
a,b,c,d = sp.symbols('a b c d')
params = a,b,c,d
function = (a*b**5*sp.sqrt(c) - d)
testd = np.array([[0.8,0.3,4.2,0,2.4,0.1,7,0.12], 
                  [6.8,6.3,4.2,6,2.4,6.1,7,6.12]])

z = pap(function, "beispielFunktion", "bspFunc", params, testd, params_without_error = [])

Results:
[[  1612.7278881     608.33459026]
 [ 13760.68704885 100696.08892025]]
gegebene Funktion:


<IPython.core.display.Math object>

beispielFunktion is at line 13
beispielFunktion is at line 17
beispielFunktion is at line 21
Die alte Funktion wurde erfolgreich überschrieben.
Formel des absoluten Fehlers der gegebenen Funktion:


<IPython.core.display.Math object>

errAbs-beispielFunktion is at line 17
Die alte Funktion wurde erfolgreich überschrieben.
Formel des relativen Fehlers der gegebenen Funktion:


<IPython.core.display.Math object>

errRel-beispielFunktion is at line 21
Die alte Funktion wurde erfolgreich überschrieben.


## Export der Werte

### Export als Tabelle

In [12]:
def process_experimental_data(path = versuchsnummer + ".csv", setDelimiter = ";", setHeader: int=0, setIndex_col: int=None, export_as_tex_tab: bool=True):
    """
    Methode, die CSV-Dateien nach gewissen regeln ausliest. Dabei werden die folgenden Eigenschaften erfuellt:
        1. Wird eine TXT-Datei importiert, so soll geschaut werden, ob diese als CSV-Datei gelesen werden kann. 

        2. Es wird gecheckt, ob eine Kopfzeile existiert, wenn ja, solld diese ordentlich ausgelesen werden.

        3. Es wird automatisiert erkannt, was der zu einem Wert gehörende Fehler ist, oder ob der Wert als fehlerfrei angenommen werden kann.

        4. Es wird automatisch erkannt, falls dieselbe Messung wiederholt wurde und mehrere Werte zu einer Messung gehoeren (Bspw. t_1, t_2 ...). 
        a. Es kann der Mittelwert bestimmt werden
        b. Es kann die Standardabweichung bestimmt werden.

        5. Der Export sowohl aller einzelnen Ergebnisse, als auch als gesamte Tabelle wird ermoeglicht.

        6. Es wird automatisch erkannt, was die Einheiten sind und 10er Potenzen koennen ermittelt werden.

        Note: Die meisten Funktionen funktionieren nur, wenn der Header ordentlich gesetzt wird und eingelesen wird.
    
    Parameter
    ---------
    **path**: Relatives verzeichnes der einzulesenden Datei. Am besten CSV-Datei einfach unter *versucshnummer.csv* im Python-Folder speichern.

    **setDelimiter**: Default ist hier >> ; << (weil Excel mal wieder lost des Grauens ist, der Bums steht ligit fuer "COMMA seperated values" ),aber Häufig ist auch >> , <<. Also schauen, wie die Spalten getrennt sind. 

    **setHeader**: Setzt fest, was die Header-Row ist. Default ist 0 (die oberste Row) 
    """

    # Liest die Daten unmaipuliert
    try:
        df = pd.read_csv(path, delimiter=setDelimiter, header=0, index_col=None)
    except FileNotFoundError:
        print(f"The file {path} was not found.")
     
    # Wir erwarten nicht, dass zwei perfekt identische Zeilen existieren koennten, daher werden alle identischen Zeilen geloescht. Somit werden auch alle NaN Zeilen entfernt    
    # Langfristig soll das noch verschönert werden und wirklich nur NaN Zeilen gelöscht werden
    no_NaN_data = df.drop_duplicates(keep= False)
    df.reset_index(drop=True, inplace=True)

    # Speichert dataFrame als Latex-Tabelle und fügt diese der python-results.sty hinzu.
    if export_as_tex_tab == True:

        pass

    return no_NaN_data 

process_experimental_data()
clean_data = process_experimental_data()


In [103]:
def table_to_latex(df_tex: str, texVarName: str, texCom: str):
    """
    Fügt aus einem DataFrame entstandene Latex Tabelle der special Python File hinzu.

    Parameters
    ----------
    **df_tex** : pd.DataFrame.to_latex
        Uebersetztes DataFrame. Kann Label und Caption handlen. Dezimaltrennung steht auf ','.

    **texVarName** : str
        Einzigartiger Name der Variablen. Dieser wird zum überschreiben alter Formel gebraucht.

    **texCom** : str
        setzt den newcommand-Kürzel für latex fest. Setzte kein Backslash! Auch texCom muss einzigartig gesetzt werden.

    Returns
    -------
    Schreibt automatisch in die sty file
    """
    
    # Hinzufuegen bzw. Ueberschreieben der Tabelle in die Sammlung
    with open(pyPath, 'r') as file:
        lines: List[str] = file.readlines()
    found = False

    with open(pyPath, 'r') as file:
        for lineNum,  line in enumerate(lines):
            if texVarName in line: # Checkt, ob der Variablen Name bereits vergeben ist.
                print(f'{texVarName} starts at line {lineNum + 1}') 
                found = True
                table_start_line = lineNum
                break

    if not found:
        print(f"Die neue Tabelle ({texVarName}) wurde hinzugefügt")
        with open(pyPath, 'w') as file:
            file.writelines(lines) # Schreibt den alten Stand
            file.write("\n% " + texVarName + "\n")
            file.write("\\newcommand{\\" + texCom + "}{" + df_tex + "}\n\n")
    else:
        print(f"Die alte Tabelle ({texVarName}) wird überschrieben.")
        start_table = table_start_line + 1
        while start_table < len(lines) and lines[start_table].strip() == "":
            # Leere Zeile überspringen – das ist unser Marker
            start_table += 1
            break   # Wir wollen nur die **erste** leere Zeile nach dem Makro

        # Falls kein leerer Marker gefunden wurde, gehen wir davon aus,
        # dass die Tabelle direkt in der nächsten Zeile startet.
        if start_table >= len(lines):
            start_table = table_start_line + 1

        end_table = start_table

        while end_table < len(lines):
            cur = lines[end_table].strip()

            # Abbruchbedingungen:
            #   Nächster Makro‑Eintrag (ein neuer \newcommand)
            #   Abschnitt / Kapitel (z. B. \section, \subsection)
            #   Beginn einer anderen Umgebung (\begin{...})
            #   Ende der Datei
            if (
                cur.startswith(r"\\newcommand")
                or cur.startswith(r"\\section")
                or cur.startswith(r"\\subsection")
                or cur.startswith(r"\\begin")
            ):
                break

            end_table += 1

        if end_table > start_table:
            del lines[start_table - 1:end_table]

        insertion_point = table_start_line + 1  # Direkt nach der Zeile mit texVarName
        new_block = [
            # "\n\n",
            f"% {texVarName}\n",
            f"\\newcommand{{\\{texCom}}}{{{df_tex}}}\n",
        ]
        lines[insertion_point:insertion_point] = new_block

        with open(pyPath, 'w') as file:
            file.writelines(lines)

In [109]:
process_experimental_data()
clean_data = process_experimental_data()

def combine_value_error(df: pd.DataFrame, show_series_values:bool=False, setTexTabCap: str="", setTexTabLab:str=""):
    """Sucht im DataFrame nach einem Asdruck:
        "<name> [<unit>] <power>"
    und dem dazugehörigem Fehlerausdruck:
        "err <name>".
    Erstellt ein neues DataFrame für den Latex-Export

    Parameters
    ----------
    **df* : pd.DataFrame
        Original data frame.

    **show_series_values** : bool
        Rather the table should show the experimental data or just its mean
        

    Returns
    -------
    pd.DataFrame
        Eine Kopie von ``df``.
    """

    # Kopiert das alte DataFrame
    df_tex = pd.DataFrame(index=df.index)


    def format_tex(row):
        val = row[col]
        err = row[err_col]
        # return f'({val} \\pm {err}) \\mathrm{{{header_unit}}}'
        return f'${val} \\pm {err}$'
    
    # ----------------------------
    # 1) Einzelmessungen
    # ----------------------------

    # Einfaches Pattern fuer Messwerte, Einheiten und Exponenten:
    pattern = re.compile(
        r'^\s*'
        r'(?P<name>[^\[]+?)'    # alles bis zur ersten '['      # Name
        r'\s*\[\s*'             # echtes '['
        r'(?P<unit>[^\]]+?)'    # alles bis zur ']'             # Einheit
        r'\s*\]'                # echtes ']'
        r'(?:\s*10\^(?P<power>-?\d+))?'                         # optionale 10^n   
        r'\s*$'
    )

    for col in df.columns:
        col_match = pattern.match(col)

        if not col_match:
            # No header that follows the expected schema – skip it
            # print(f'Skipping column {col!r}: not "<value> [<unit>] <power>"')
            continue

        # Extract the captured parts
        name = col_match.group('name').strip()
        unit = col_match.group('unit').strip()
        power = col_match.group("power")   
        # print(f'Found measurement column: name={name!r}, unit={unit!r}, power={power!r}')

        # Build the name of the error column that must exist
        err_col = f'err {name}'
        if err_col not in df.columns:
            # print(f'No error column "{err_col}" for measurement "{name}" - skipping')
            continue

        if power:
            header_unit = f"$\\mathrm{{{unit}}} \\cdot 10^{{{power}}}$"
        else:
            header_unit = f"$\\mathrm{{{unit}}}$"

        df_tex[f'{name} [{header_unit}]'] = df.apply(format_tex, axis=1)


    # ----------------------------
    # 2) Messreihen
    # ----------------------------

    # Schema fuer Messreihen
    series_pattern = re.compile(
        r'^(?P<base>\w+)_\d+'
        r'(?:\s*\[\s*(?P<unit>[^\]]+)\s*\])?'
    )

    grouped = {}

    for col in df.columns:
        series_match = series_pattern.match(col)
        if series_match:
            base = series_match.group("base")
            unit = series_match.group("unit")
            # power = col_match.group("power")
            grouped.setdefault(base, {"cols": [], "unit": unit})
            grouped[base]["cols"].append(col)


    # --- Berechnungen ---
    for base, info in grouped.items():
        cols = info["cols"]
        series_unit = info["unit"] or ""


        err_col = f"err {base}"
        if err_col not in df.columns:
            continue

        values = df[cols]

        if show_series_values == True:
            # Einzelwerte in DataFrame hinzufuegen
            for ind, col in enumerate(cols, start=1):
                df_tex[f'{base}_{ind} [{unit}]'] = df.apply(format_tex, axis=1)

        # Mittelwert
        mean = values.mean(axis=1)

        # Standardabweichung
        std = values.std(axis=1, ddof=1)

        # Fehler des Mittelwerts
        stat_err = std / np.sqrt(len(cols))

        # systematischer Fehler
        sys_err = df[err_col]

        # Gesamtfehler
        total_err = np.sqrt(stat_err**2 + sys_err**2)

        # if power:
        #     header_unit = f"\\mathrm{{{unit}}} \\cdot 10^{{{power}}}"
        # else:
        #     header_unit = f"\\mathrm{{{unit}}}"

        df_tex[f"$\\overline{{{base}}} [\\mathrm{{{series_unit}}}]$"] = (
            "$" +
            mean.round(3).astype(str) + 
            " \\pm " + 
            total_err.round(3).astype(str)+
            "$"
        )

    setTexTabCap="Dies ist die Caption der Tabelle"
    setTexTabLab="bspTab"

    table_to_latex(df_tex.to_latex(decimal=',', caption=setTexTabCap, label=setTexTabLab, position="h", index=False), texVarName="Beispielhafet Tabelle", texCom="bspTab")

    print(df_tex.to_latex(decimal=',', caption=setTexTabCap, label=setTexTabLab, position="h"))
    return df_tex.to_latex(decimal=',', caption=setTexTabCap, label=setTexTabLab, position="h")


combine_value_error(clean_data)

Beispielhafet Tabelle starts at line 12
Die alte Tabelle (Beispielhafet Tabelle) wird überschrieben.
\begin{table}[h]
\caption{Dies ist die Caption der Tabelle}
\label{bspTab}
\begin{tabular}{llll}
\toprule
 & Value [$\mathrm{m}$] & Wert [$\mathrm{V} \cdot 10^{12}$] & $\overline{t} [\mathrm{s}]$ \\
\midrule
0 & $5.0 \pm 0.7$ & $0.378 \pm 0.0056894$ & $160.667 \pm 73.334$ \\
1 & $7.0 \pm 0.6$ & $0.8564 \pm 0.0056894$ & $596.667 \pm 29.68$ \\
2 & $6.0 \pm 0.7$ & $0.68 \pm 0.0056894$ & $3003.0 \pm 2436.0$ \\
3 & $7.0 \pm 0.14$ & $0.6894 \pm 0.0056894$ & $567.0 \pm 0.5$ \\
\bottomrule
\end{tabular}
\end{table}



'\\begin{table}[h]\n\\caption{Dies ist die Caption der Tabelle}\n\\label{bspTab}\n\\begin{tabular}{llll}\n\\toprule\n & Value [$\\mathrm{m}$] & Wert [$\\mathrm{V} \\cdot 10^{12}$] & $\\overline{t} [\\mathrm{s}]$ \\\\\n\\midrule\n0 & $5.0 \\pm 0.7$ & $0.378 \\pm 0.0056894$ & $160.667 \\pm 73.334$ \\\\\n1 & $7.0 \\pm 0.6$ & $0.8564 \\pm 0.0056894$ & $596.667 \\pm 29.68$ \\\\\n2 & $6.0 \\pm 0.7$ & $0.68 \\pm 0.0056894$ & $3003.0 \\pm 2436.0$ \\\\\n3 & $7.0 \\pm 0.14$ & $0.6894 \\pm 0.0056894$ & $567.0 \\pm 0.5$ \\\\\n\\bottomrule\n\\end{tabular}\n\\end{table}\n'

# Back Up erstellen

In [14]:
def placeholder():
    """
    Erklärung der Methode
    """
    return